Data set downloaded off the https://my.usgs.gov/bpd/search/search/search website.
Data needed to be downloaded piecemeal to avoid truncation at 10000 samples. After some refinement, 1 year increments was selected as the download size.
Data downloaded is in captures only, as these are the most likely to be accurate species differentiations, and should be reflective of actual population values.

In [ ]:
#Original Test code, non-iterative for prototying purposes.
import pandas as pd

def get_data(start_year,end_year):
    string=str("https://my.usgs.gov/bpd/search/search/search?format=csv&extension=csv&country=240&gadmLevel0=240&suborder=All&toyear_year="+ end_year +"&fromyear=date.struct&countryFips=240&SYNCHRONIZER_URI=%2Fbpd%2Fsearch%2Fsearch%2Fsearch&dataProjects=All&search=Perform+Search&fromyear_year=" + start_year + "&what=captures&genus=All&species=All&state=All&speciesOnly=true&family=All&toyear=date.struct&SYNCHRONIZER_TOKEN=4145915f-031c-4173-9256-0c8f4c92341f")
    x=pd.read_csv(string)
    return x

test= get_data("1935", "1933")


In [ ]:
## Importing data iteratively
import pandas as pd

def get_data(start_year,end_year):
    """
    End year is inclusive.
    """
    full_data=[]
    from_year=start_year
    to_year=from_year+1
    while from_year <= end_year:
        str_from=str(from_year)
        str_to=str(to_year)
        string="https://my.usgs.gov/bpd/search/search/search?format=csv&extension=csv&country=240&gadmLevel0=240&suborder=All&toyear_year="+ str_to +"&fromyear=date.struct&countryFips=240&SYNCHRONIZER_URI=%2Fbpd%2Fsearch%2Fsearch%2Fsearch&dataProjects=All&search=Perform+Search&fromyear_year=" + str_from + "&what=captures&genus=All&species=All&state=All&speciesOnly=true&family=All&toyear=date.struct&SYNCHRONIZER_TOKEN=4145915f-031c-4173-9256-0c8f4c92341f"
        x=pd.read_csv(string)
        full_data.append(x)
        print("Got a year!")
        from_year += 1
        to_year += 1

    return full_data
        
test= get_data(1918, 2018)
print(test)

In [ ]:
df_n.to_csv("Bat_Data_1918_to_2018.csv")

In [ ]:
print(type(df_n))
print(df_n.columns)
print(test[0].columns)

In [1]:
#### This is the Initial Cleaning of Data ########
import pandas as pd
import numpy as np
bat_db=pd.read_csv(r"C:\Users\Mariko\Documents\GitHub\Bat_Project\Bat_Data_1918_to_2018.csv")
#print(bat_db.head())

#Useful metrics:Date, State, County, Species (13 unknown),Data Project
#So-So metrics:Site(often descriptors such as "near creek in Earlham Cemetary"),Sex (22614 unknown), Age (234410 unknown)
#Not useful metrics: Reproduction (253964 unknown), Forearm(23 known),Weight(85 known)


db_clean=bat_db.drop(["Unnamed: 0", "Reproduction", "Forearm","Weight", "Site"], axis=1)
db_clean.head()

#Seperating the Year and Month Columns
db_clean["Year"]=db_clean["Date"].str[0:4].astype(int)
db_clean["Month"]=db_clean["Date"].str[5:7].astype(int)
db_clean["Day"]=db_clean["Date"].str[8:10].astype(int)
db_clean["Time"]=db_clean["Date"].str[10:]
print(db_clean['Time'].value_counts(dropna=False))
    #All time values= 00:00:00.0, dropping time column
db_clean=db_clean.drop("Time", axis=1)

#Changing "Unknown" to NaN
print(db_clean['Sex'].value_counts(dropna=False))
db_clean["Sex"]=db_clean["Sex"].replace("Unknown", np.nan)
#df['range'] = df['range'].str.replace(',','-')
print(db_clean['Sex'].value_counts(dropna=False))

db_clean.head()

 00:00:00.0    275464
Name: Time, dtype: int64
Female     140547
Male       112300
Unknown     22615
NaN             2
Name: Sex, dtype: int64
Female    140547
Male      112300
NaN        22617
Name: Sex, dtype: int64


,Id,Date,County,State,Species,Sex,Age,Data Project,Year,Month,Day
0,2568572,1921-01-02 00:00:00.0,"Carter, US.KY",Kentucky,Myotis lucifugus,NaN,Unknown,U.S. Bat Banding Program Data,1921,1,2
1,2702233,1933-07-22 00:00:00.0,"Barnstable, US.MA",Massachusetts,Myotis lucifugus,NaN,Unknown,U.S. Bat Banding Program Data,1933,7,22
2,2702234,1933-07-22 00:00:00.0,"Barnstable, US.MA",Massachusetts,Myotis lucifugus,NaN,Unknown,U.S. Bat Banding Program Data,1933,7,22
3,2702235,1933-07-22 00:00:00.0,"Barnstable, US.MA",Massachusetts,Myotis lucifugus,NaN,Unknown,U.S. Bat Banding Program Data,1933,7,22
4,2702236,1933-07-22 00:00:00.0,"Barnstable, US.MA",Massachusetts,Myotis lucifugus,NaN,Unknown,U.S. Bat Banding Program Data,1933,7,22


In [5]:
#### Visualization and analysis ########
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
import pandas as pd

def bats_per_state (dataframe, state, unknown="y", graph="y"):
    """
    Imput a state, and this function will generate a list of each bat species and the number of 
    bats captured for each species. If you want to exclude unknown species, set unknown=n.
    Returns a dataframe.
    """
    bat_state= dataframe.loc[(dataframe["State"]==state), "Species"]
    b_spec_count={}
    for i in bat_state:
        if i not in b_spec_count:
            b_spec_count[i]=1
        else:
            b_spec_count[i]+=1
    if unknown=="n":
        del(b_spec_count["Unknown species"])
    if  bool(b_spec_count)==False:
        return "No data available for that state."
    else:
        b_df=pd.DataFrame.from_dict(b_spec_count, orient='index', dtype=None)
        b_df.columns=["# Observed"]
        b_df.sort_values("# Observed",inplace=True,ascending=False)
    if graph=="y":
        num_colour=b_df.shape[0]
        dark_bat=sns.cubehelix_palette(n_colors=num_colour,light=0.75, reverse=True)
        b=sns.barplot(x=b_df.index, y=np.log10(b_df["# Observed"]),data=b_df, palette=dark_bat)
        sns.set_style(style='white')
        plt.xticks(rotation=90)
        plt.ylabel("Log(# Observed)")
        return b_df
    else:
        return b_df
        

def bats_pop_time (dataframe, state, species="all", year=[1918,2018], unknown="y", graph="y"):
    """
    Imput a state and years of inquiry, and this function will generate 
    the number of bats of that species captured in that year. Leave year or state blank to search all
    states and/or years. If you want to exclude unknown species, set unknown=n.
    Returns a Pandas Dataframe, and a graph, if desired.
    """
    db_state=dataframe[dataframe.State.isin(state)]
    if year[0]>=year[1]:
        raise Exception("Please enter a list consisting of a start year, followed by an end year. They cannot be the same year")
    time=list(range(year[0],year[1]))
        #This creates a list of years between the start and end years
    db_time=db_state[db_state.Year.isin(time)] 
        #This produces a dataframe that has been filtered by both state and time
    return db_time




In [ ]:
###Quick-test function: bats_per_state  ###
p=bats_per_state(db_clean,"Oregon")
print(p)

In [ ]:
###Quick-test function: bats_pop_time  ###
tst_state=bats_pop_time(db_clean, ["Ohio", "California"], year=[1945, 1949])
print(tst_state)   